In [1]:
from openai import OpenAI, AsyncOpenAI
import asyncio
import os
from datasets import load_dataset
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
from tqdm import tqdm   
import emoji
import json
from evaluate import load
# from utils import get_dataset
import numpy as np
bertscore = load("bertscore")


2024-04-21 20:30:51.817258: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 20:30:51.817487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 20:30:51.889127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-21 20:30:52.026093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 20:30:53.660504: W tensorflow/compiler/tf2

In [2]:
import os
import re
from bisect import bisect_left
from collections import defaultdict
import pandas as pd
import numpy as np
import math
from datasets import load_dataset, load_from_disk
import json
import random
import re
from evaluate import load

  



## Few-shots + explicit_errors
few_shots_explicit = '''You are given a few examples of erroneous sentences and their corrections. For every example, you are given the explicit errors and their corrections. You are also given an erroneous input sentence and its correction, followed by the explicit errors in this sentence. Output a simple explanation for each error in the erroneous sentence.'''

## Few shots + no explicit errors
few_shots_no_explicit = '''You are given a few examples of erroneous sentences and their corrections. You are also given an erroneous input sentence and its correction. Output a simple explanation for each error in the erroneous sentence.'''

## No few-shots + explicit
no_few_shots_explicit = '''You are given an erroneous input sentence and its correction, followed by the explicit errors in this sentence. Output a simple explanation for each error in the erroneous sentence.'''

## No few-shots + no explicit
no_few_shots_no_explicit = '''You are given an erroneous input sentence and its correction. Output a simple explanation for each error in the erroneous sentence.'''



def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

def put_explicit_error(example):
    ret = ''
    ret += f"Errors:\n"

    for i,correction in enumerate(example['corrections']):
        correct_word = ' ' if correction['correct'] == '-' else correction['correct']
        error_word = ' ' if correction['error'] == '-' else correction['error']

        ret += f"{i+1}. Error: {error_word}, Correction: {correct_word}\n"
    
    return ret


def generate_prompt(example, prompt_head = None,num_shots=0, explicit_errors = 0,train = 1, dataset = None, field = None):
    full_prompt  = prompt_head

    # print(example.keys())

    if num_shots:
        idx= np.random.randint(0,len(dataset),num_shots)
        samples = dataset.select(idx)

        for sample in samples:
            full_prompt += f"Erroneous sentence: {sample['incorrect_sentence']}\n"
            full_prompt += f"Correct sentence: {sample['correct_sentence']}\n"
            if explicit_errors:
                full_prompt += put_explicit_error(sample)

            full_prompt += f"Explanations:\n"
            for i,correction in enumerate(sample['corrections']):
                full_prompt += f"{i+1}. {correction['explanation']}\n"
            
    full_prompt += f"Erroneous sentence: {example['incorrect_sentence']}\n"
    full_prompt += f"Correct sentence: {example['correct_sentence']}\n"

    if explicit_errors:
        full_prompt += put_explicit_error(example)
        
    full_prompt += f"Explanations:\n"

    labels = ''
    for i,correction in enumerate(example['corrections']):
        labels += f"{i+1}. {correction['explanation']}\n"
    if train:
            full_prompt += labels


    # example['text'] = 'sd'
    example[field] = full_prompt.strip()
    example['label'] = labels
    # print(example.keys())

    return example

def get_dataset(dataset_path, split='train', field='prompt', num_shots=0,explicit_errors = 0):
    

    # Load the dataset
    dataset = load_dataset(dataset_path)[split]

    ########### Choose the prompt head based on the options ###########
    if num_shots and explicit_errors:
        prompt_head = few_shots_explicit
    elif num_shots and not explicit_errors:
        prompt_head = few_shots_no_explicit
    elif not num_shots and explicit_errors:
        prompt_head = no_few_shots_explicit
    else:
        prompt_head = no_few_shots_no_explicit

    dataset = dataset.map(generate_prompt, fn_kwargs={"field": field, \
        "prompt_head": prompt_head, "train": split == 'train', \
        'num_shots': num_shots,'dataset':dataset , 'explicit_errors': explicit_errors})
        
    return dataset

In [3]:
DATASET = "boda/kaneko_data"
N_SHOTS = 1
EXPLICIT = 1

val_dataset = get_dataset(
    dataset_path = DATASET,
    split='test',
    field='prompt',
    num_shots=N_SHOTS,
    explicit_errors =EXPLICIT)


Map:   0%|          | 0/144 [00:00<?, ? examples/s]

In [6]:
print(val_dataset[1]['prompt'])

You are given a few examples of erroneous sentences and their corrections. For every example, you are given the explicit errors and their corrections. You are also given an erroneous input sentence and its correction, followed by the explicit errors in this sentence. Output a simple explanation for each error in the erroneous sentence.Erroneous sentence: Therefore , I thought a known genetic risk should not be obligated to tell his or her family or relatives .
Correct sentence: Therefore , I think a known genetic risk should not be obligated to tell his or her family or relatives .
Errors:
1. Error: thought, Correction: think
Explanations:
1. You are expressing an opinion that you hold now, not something that you thought in the past, so you should use the present tense, "think".
Erroneous sentence: On the other side , as stated above , a person who want to confirm and has been prepared to face the result can choose to do the genetic testing individually .
Correct sentence: On the other

In [5]:
model_name = "gpt-3.5-turbo-0125"	
outputs = []
labels = []
for example in tqdm(val_dataset):
    
    prompt = example['prompt']

    response = None
    
    try:
      # # correct_answers.append(clue["target"])
      # clue_message = {"role": "user", "content":prompt}
      # completion = client.chat.completions.create(
      #   model=model_name,
      #   # response_format={"type" : "json_object"}, ## Add jsno output
      #   messages=[
      #     # system_message,
      #     clue_message
      #   ]
      # )

      # response = completion.choices[0].message.content.lower()
      # outputs.append(response)
      labels.append(example["label"])
    except:
      
      print("Error")
  



100%|██████████| 144/144 [00:00<00:00, 4628.37it/s]


In [12]:
for i in range(5):
    print(val_dataset[i]['incorrect_sentence'])
    print(val_dataset[i]['correct_sentence'])
    print(labels[i])
    print("\n")

Therefore , the lack of creative qualified human resource seriously hampers engineering design processes for innovations in China .
Therefore , the lack of creative human resources seriously hampers engineering design processes for innovations in China .
1. The adjective "qualified" means having the necessary skills to perform a particular task, but it refers mainly to official or certified qualifications, such as tests. There is no test that would certify someone's creative thinking abilities, so "qualified" should be omitted. The sentence could also be reworded to "human resources with creative thinking abilities," but "creative human resources" expresses the same idea more concisely and with less potential for confusion.
2. The phrase "human resources" refers to a group of people who are able to do work. Because it refers to a group, it is only used in the plural in principle. 



On the other side , as stated above , a person who want to confirm and has been prepared to face the re

In [33]:
output_path = f"outputs/{model_name}_shots={N_SHOTS}_explicit={EXPLICIT}.txt"
with open(output_path, "w") as f:
        for output in outputs:
            f.write(output + "\n")

In [34]:
len(outputs), len(labels)


(144, 144)

In [35]:
metrics = bertscore.compute(predictions=outputs, references=labels, lang="en")

f1 = np.array(metrics['f1']).mean()
preceision = np.array(metrics['precision']).mean()
recall = np.array(metrics['recall']).mean()

print(f'Model: {model_name}')
print('F1:', f1)
print('Precision:', preceision)
print('Recall:', recall)
save_file = 'outputs/' + model_name + '_metrics.txt'
with open(save_file, 'w') as f:
    f.write(f"BERTScore: \n F1: {f1} \n Precision: {preceision} \n Recall: {recall} \n")


Model: gpt-3.5-turbo-0125
F1: 0.8923587881856494
Precision: 0.8960178229543898
Recall: 0.889106560498476


In [ ]:
from evaluate import load
bertscore = load("bertscore")




In [1]:
labeles = []
predictions = []

output_path = 'llama3_outputs/Meta-Llama-3-8B_outputs_5-shots_False-explicit.txt'
lines = []
with open(output_path, 'r') as f:
    lines = f.readlines()

import re
s = ''
for i in range(0, len(lines)):
    l = lines[i]
    if re.match(r'^\d+\.', l):
        print(l)

# metrics = bertscore.compute(predictions=outputs, references=labels, lang="en")



1. This is a spelling error. The correct spelling is "showing."

2. The noun "experience" can be either countable or uncountable, depending on its meaning. It is countable when it refers to events in a person's life and It is uncountable when it refers to work experience or skills or knowledge gained by doing something. intangible things. In the context of sharing photos on social media, it means life events, so it is countable. It should be made plural because the photos show multiple events in the lives of different people.

1. Incorrect word choice. When talking about reading something, you would say "read on" when the information is

2. The phrase "human resources" refers to a group of people who are able to do work. Because it refers to a group, it is only used in the plural in principle. 

2. "Want" is a verb, not a noun, so you need to use the verb form here.

3. "Has been prepared" is a passive construction, so you need to use the passive form of the verb "to choose".

1. The s